# Atelier 2 : Prédiction Fonctionnelle avec Tax4Fun2

Cet atelier est complémentaire à l'atelier FUNGuild. Nous partons ici des fichiers de sortie de QIIME2 (données 16S) pour réaliser une analyse fonctionnelle des communautés **procaryotes** (bactéries et archées) à l'aide de l'outil Tax4Fun2.

## Introduction

Après avoir analysé les communautés fongiques avec FUNGuild, nous allons maintenant explorer les **fonctions métaboliques** des bactéries et archées présentes dans nos échantillons de sol.

**Tax4Fun2** est un outil qui prédit les fonctions métaboliques des communautés procaryotes en se basant sur leur taxonomie (séquences 16S rRNA). Il utilise des bases de données de génomes de référence pour inférer le contenu génétique et les voies métaboliques (KEGG pathways) présentes dans la communauté.

### Objectifs de l'atelier :
1. Installer et configurer Tax4Fun2 dans Google Colab.
2. Préparer les données issues de QIIME2 pour Tax4Fun2.
3. Exécuter Tax4Fun2 pour prédire les fonctions métaboliques.
4. Visualiser les voies métaboliques (KEGG pathways) par échantillon.
5. Corréler les fonctions avec les propriétés du sol.

### Différences avec FUNGuild :
- **FUNGuild** : Guildes écologiques de champignons (ITS)
- **Tax4Fun2** : Fonctions métaboliques de procaryotes (16S)
- **FUNGuild** : Rapide, basé sur une base de données de guildes
- **Tax4Fun2** : Plus lent, utilise des génomes de référence pour prédiction

## Étape 1 : Installation de Tax4Fun2 et téléchargement des données

Tax4Fun2 est un package R. Nous allons l'installer dans Google Colab avec toutes ses dépendances.

In [ ]:
print('📦 Installation de Tax4Fun2 et dépendances...')
print('⏱️  Cette étape peut prendre 15-20 minutes (ou plus)')
print('='*60)

# Instalar R e dependências do sistema
print('\n📥 Installation de R et des dépendances système...')
!apt-get update -qq > /dev/null 2>&1
!apt-get install -y r-base r-base-dev libcurl4-openssl-dev libssl-dev libxml2-dev libgit2-dev > /dev/null 2>&1

print('✅ R installé')

# Instalar rpy2 para integração Python-R
print('\n📦 Installation de rpy2 pour l\'intégration Python-R...')
!pip install -q rpy2

print('✅ rpy2 installé')

# Carregar rpy2
import rpy2.robjects as robjects
from rpy2.robjects import r

print('\n📦 Installation des packages R requis...')

# Instalar packages R
r('''
options(repos = c(CRAN = "https://cloud.r-project.org/"))
install.packages(c("devtools", "Rcpp", "ggplot2", "dplyr", "tidyr", "seqinr", "biomformat"),
                 quiet = TRUE, dependencies = TRUE)
''')

print('✅ Packages R de base installés')

# Instalar Tax4Fun2
print('\n📦 Installation de Tax4Fun2...')
r('''
library(devtools)
install_github("fjossandon/Tax4Fun2", quiet = TRUE)
''')

# Verificar instalação
r('''
if (require("Tax4Fun2", quietly = TRUE)) {
    print("✅ Tax4Fun2 chargé avec succès!")
} else {
    stop("❌ ERREUR : Tax4Fun2 n'a pas pu être chargé")
}
''')

print('\n✅ Installation terminée avec succès!')

# Clonage du dépôt GitHub
print('\n📥 Clonage du dépôt de données...')
!git clone https://github.com/thgum/AtelierSYMBIOSOL.git

print('\n📂 Fichiers de données disponibles :')
!ls -lh AtelierSYMBIOSOL/data/data_Tax4Fun2/

📦 Installation de Tax4Fun2 et dépendances...
⏱️  Cette étape peut prendre 15-20 minutes (ou plus)

📥 Installation de R et des dépendances système...
✅ R installé

📦 Installation de rpy2 pour l'intégration Python-R...
✅ rpy2 installé


(as ‘lib’ is unspecified)




📦 Installation des packages R requis...


In [ ]:
# Clonage du dépôt GitHub pour accéder aux données
print('📥 Clonage du dépôt de données...')
!git clone https://github.com/thgum/AtelierSYMBIOSOL.git

# Lister les fichiers pour vérifier
print('\n📂 Fichiers de données disponibles pour Tax4Fun2 :')
!ls -lh AtelierSYMBIOSOL/data/data_Tax4Fun2/

## Étape 2 : Préparation des fichiers d'entrée

Tax4Fun2 nécessite deux fichiers principaux :
1. **Fichier FASTA** : Séquences représentatives des ASVs/OTUs
2. **Table d'abondance** : Abondances des ASVs par échantillon

Nous avons ces fichiers depuis QIIME2. Vérifions leur format.

In [5]:
import pandas as pd
import os

# Définir les chemins
data_dir = 'AtelierSYMBIOSOL/data/data_Tax4Fun2'
fasta_file = os.path.join(data_dir, 'dna-sequences.fasta')
otu_file = os.path.join(data_dir, 'otu_table.txt')
tax_file = os.path.join(data_dir, 'taxonomy.tsv')

print('📊 Vérification des fichiers d\'entrée...')
print('='*60)

# Vérifier le fichier FASTA
if os.path.exists(fasta_file):
    with open(fasta_file, 'r') as f:
        fasta_lines = f.readlines()
    n_sequences = sum(1 for line in fasta_lines if line.startswith('>'))
    print(f'\n✅ Fichier FASTA : {n_sequences} séquences')
    print(f'   Chemin : {fasta_file}')
else:
    print(f'\n❌ Fichier FASTA non trouvé : {fasta_file}')

# Charger la table d'OTUs
if os.path.exists(otu_file):
    otu_table = pd.read_csv(otu_file, sep='\t', skiprows=1)
    print(f'\n✅ Table d\'OTUs : {otu_table.shape[0]} ASVs × {otu_table.shape[1]} colonnes')
    print(f'   Nombre d\'échantillons : {otu_table.shape[1] - 1}')
    print('\n📋 Aperçu de la table d\'OTUs :')
    display(otu_table.head())
else:
    print(f'\n❌ Table d\'OTUs non trouvée : {otu_file}')

# Charger la taxonomie
if os.path.exists(tax_file):
    tax_table = pd.read_csv(tax_file, sep='\t', skiprows=1, header=None,
                           names=['Feature ID', 'Taxon', 'Confidence'])
    print(f'\n✅ Table de taxonomie : {tax_table.shape[0]} taxons')
    print('\n📋 Aperçu de la taxonomie :')
    display(tax_table.head())
else:
    print(f'\n❌ Table de taxonomie non trouvée : {tax_file}')

📊 Vérification des fichiers d'entrée...

✅ Fichier FASTA : 13466 séquences
   Chemin : AtelierSYMBIOSOL/data/data_Tax4Fun2/dna-sequences.fasta

✅ Table d'OTUs : 13466 ASVs × 76 colonnes
   Nombre d'échantillons : 75

📋 Aperçu de la table d'OTUs :


,#OTU ID,LEM59-01,LEM59-02,LEM59-03,LEM59-04,LEM59-05,LEM59-06,LEM59-07,LEM59-08,LEM59-09,...,LEM59-66,LEM59-67,LEM59-68,LEM59-69,LEM59-70,LEM59-71,LEM59-72,LEM59-73,LEM59-74,taxonomy
0,8497e8776933fb43f86a84ad1358fd91,2081.0,1912.0,118.0,97.0,1294.0,41.0,17.0,198.0,1458.0,...,1730.0,521.0,3487.0,5914.0,1542.0,8276.0,724.0,4612.0,4106.0,k__Bacteria; p__Actinobacteria; c__Actinobacte...
1,e6fa43883a30a05c1bf46fd28fa2067d,1687.0,1618.0,242.0,157.0,1056.0,71.0,876.0,7187.0,9381.0,...,10236.0,6174.0,9376.0,7158.0,3052.0,7052.0,2458.0,2212.0,2047.0,k__Bacteria; p__Actinobacteria; c__Actinobacte...
2,7b08de2adb95b866362152360da56d53,64.0,0.0,26.0,40.0,136.0,33.0,47.0,0.0,0.0,...,5658.0,8485.0,4356.0,1776.0,8500.0,6981.0,4268.0,4352.0,4001.0,k__Bacteria; p__Proteobacteria; c__Betaproteob...
3,1a1e74c6c7cacc00f00e83cb282363aa,0.0,87.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2711.0,1859.0,463.0,326.0,2051.0,1762.0,4152.0,927.0,318.0,k__Bacteria; p__Proteobacteria; c__Betaproteob...
4,0ca1ef9b5ffcd599c7d47f8bb65e5784,680.0,571.0,780.0,929.0,1118.0,719.0,353.0,384.0,512.0,...,726.0,1712.0,512.0,404.0,1243.0,797.0,615.0,939.0,945.0,k__Bacteria; p__Proteobacteria; c__Alphaproteo...



✅ Table de taxonomie : 13466 taxons

📋 Aperçu de la taxonomie :


,Feature ID,Taxon,Confidence
0,8497e8776933fb43f86a84ad1358fd91,k__Bacteria; p__Actinobacteria; c__Actinobacte...,0.939092
1,e6fa43883a30a05c1bf46fd28fa2067d,k__Bacteria; p__Actinobacteria; c__Actinobacte...,0.972020
2,7b08de2adb95b866362152360da56d53,k__Bacteria; p__Proteobacteria; c__Betaproteob...,0.898718
3,1a1e74c6c7cacc00f00e83cb282363aa,k__Bacteria; p__Proteobacteria; c__Betaproteob...,0.938572
4,0ca1ef9b5ffcd599c7d47f8bb65e5784,k__Bacteria; p__Proteobacteria; c__Alphaproteo...,0.999556


### Préparer les fichiers pour Tax4Fun2

Tax4Fun2 nécessite un format spécifique. Nous allons créer les fichiers nécessaires.

In [6]:
# Créer un répertoire pour les résultats
!mkdir -p tax4fun2_output

# Préparer la table d'abondance (format requis par Tax4Fun2)
# Tax4Fun2 attend : première colonne = ID, autres colonnes = échantillons
otu_for_tax4fun = otu_table.copy()

# Sauvegarder dans le format correct
otu_for_tax4fun.to_csv('tax4fun2_output/otu_table_for_tax4fun2.txt',
                       sep='\t', index=False)

# Copier le fichier FASTA
!cp {fasta_file} tax4fun2_output/sequences.fasta

print('✅ Fichiers préparés pour Tax4Fun2 :')
!ls -lh tax4fun2_output/

✅ Fichiers préparés pour Tax4Fun2 :
total 8.9M
-rw-r--r-- 1 root     root 5.3M Oct 27 04:18 otu_table_for_tax4fun2.txt
drwxr-xr-x 2 root     root 4.0K Oct 27 04:13 Ref100R
drwxr-xr-x 2 root     root 4.0K Oct 27 02:14 results
-rw-r--r-- 1 root     root 3.5M Oct 27 04:18 sequences.fasta
drwxr-xr-x 8 15039045 5000 4.0K Oct 27 04:14 Tax4Fun2_ReferenceData_v2
drwxr-xr-x 2 root     root 4.0K Oct 27 03:29 tax4fun2_results


## Étape 3 : Téléchargement de la base de données de référence

Tax4Fun2 nécessite une base de données de génomes de référence. Nous allons télécharger la base de données Ref100NR (version compacte).

In [7]:
print('📥 Téléchargement de la base de données Tax4Fun2 depuis Zenodo...')
print('⏱️  Fichier : 2.1 GB - Cette étape peut prendre 10-30 minutes')
print('='*60)

# Download direto do Zenodo (link estável)
#!wget -O tax4fun2_output/Tax4Fun2_ReferenceData_v2.tar.gz \
#  https://zenodo.org/records/10035668/files/Tax4Fun2_ReferenceData_v2.tar.gz

#print('\n📦 Décompression de la base de données...')
#!cd tax4fun2_output && tar -xzf Tax4Fun2_ReferenceData_v2.tar.gz

#print('\n🧹 Nettoyage...')
#!rm tax4fun2_output/Tax4Fun2_ReferenceData_v2.tar.gz

#print('\n✅ Base de données téléchargée et extraite!')

# Verificar integridade usando R
#print('\n🔍 Vérification de l\'intégrité de la base de données...')

#import rpy2.robjects as robjects
#r = robjects.r

#r('''
#library(Tax4Fun2)
#tryCatch({
#    testReferenceData(path_to_reference_data = "tax4fun2_output/Tax4Fun2_ReferenceData_v2")
#    print("✅ Base de données vérifiée et prête!")
#}, error = function(e) {
#    print(paste("⚠️  Vérification échouée:", e$message))
 #   print("   (Cela peut être normal, continuez avec l'analyse)")
#})
#''')

#print('\n📂 Contenu du répertoire:')
#!ls -lh tax4fun2_output/

📥 Téléchargement de la base de données Tax4Fun2 depuis Zenodo...
⏱️  Fichier : 2.1 GB - Cette étape peut prendre 10-30 minutes


## Étape 4 : Exécution de Tax4Fun2

Maintenant nous allons exécuter Tax4Fun2. Cette étape peut prendre **10-20 minutes** selon le nombre d'ASVs.

Tax4Fun2 effectue les étapes suivantes :
1. **Assignation taxonomique** des ASVs contre la base de référence
2. **Prédiction du contenu génétique** basé sur les génomes de référence
3. **Calcul des abondances** des voies métaboliques KEGG

In [8]:
# --- Étape 1: Installation des dépendances R et de Tax4Fun2 ---
#print("🔧 Installation de R et des dépendances...")
#!apt-get install -y r-base r-base-dev
#!pip install -q rpy2==3.5.5  # Installe l'interface Python-R

# Configuration de l'environnement R
#import os
#os.environ['R_HOME'] = '/usr/lib/R'
#import rpy2.robjects as robjects
#from rpy2.robjects.packages import importr

# Installation de Tax4Fun2 via BiocManager
#print("\n📦 Installation de Tax4Fun2 (cela peut prendre quelques minutes)...")
#robjects.r('''
#    if (!requireNamespace("BiocManager", quietly = TRUE))
#        install.packages("BiocManager")
#    if (!requireNamespace("Tax4Fun2", quietly = TRUE))
#        BiocManager::install("Tax4Fun2", update=FALSE, ask=FALSE)
#''')
#print("✅ Tax4Fun2 est installé.")
## --- Étape 2: Téléchargement de la base de données de référence ---
#ref_db_path = "tax4fun2_output"
#if not os.path.exists(ref_db_path):
#    os.makedirs(ref_db_path)
#    print("\n🌐 Téléchargement de la base de données de référence (Fichier volumineux, ~8 Go)...")
#    !wget -q -P tax4fun2_output/ https://bioinfo.mpi-inf.mpg.de/projects/Tax4Fun2/Tax4Fun2_ReferenceData_v2.tar.gz
#    print("\n📦 Décompression de la base de données...")
#    !cd tax4fun2_output && tar -xzf Tax4Fun2_ReferenceData_v2.tar.gz
#else:
#    print("\n✅ La base de données de référence existe déjà.")


In [9]:
#!apt-get update -qq
#!apt-get install -y ncbi-blast+

In [10]:
!blastn -version

blastn: 2.12.0+
 Package: blast 2.12.0, build Mar  8 2022 16:19:08


In [1]:
!apt-get update -qq
!apt-get install -y ncbi-blast+   # BLAST+ para o Tax4Fun2 (usa blastn)
!pip install -q rpy2              # ponte Python↔R

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  ncbi-data
The following NEW packages will be installed:
  ncbi-blast+ ncbi-data
0 upgraded, 2 newly installed, 0 to remove and 38 not upgraded.
Need to get 15.8 MB of archives.
After this operation, 71.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ncbi-data all 6.1.20170106+dfsg1-9 [3,519 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ncbi-blast+ amd64 2.12.0+ds-3build1 [12.3 MB]
Fetched 15.8 MB in 1s (13.2 MB/s)
Selecting previously unselected package ncbi-data.
(Reading database ... 126718 files and directories currently installed.)
Preparing to unpack .../ncbi-data_6.1.20170

In [16]:
!pip show rpy2

Name: rpy2
Version: 3.5.5
Summary: Python interface to the R language (embedded R)
Home-page: https://rpy2.github.io
Author: Laurent Gautier
Author-email: lgautier@gmail.com
License: GPLv2+
Location: /usr/local/lib/python3.12/dist-packages
Requires: cffi, jinja2, pytz, tzlocal
Required-by: 


[1] 0


In [18]:
%%R
# --- CONFIGURAÇÃO DE CAMINHOS ---
# Ajuste estes caminhos conforme os arquivos que você tem em /content
otu_tsv      <- "/content/tax4fun2_output/otu_table_for_tax4fun2.txt"        # sua OTU table em TSV
fasta_file   <- "/content/tax4fun2_output/sequences.fasta"          # seu FASTA (seqs representativas)
ref_db_root  <- "/content/tax4fun2_output/Tax4Fun2_ReferenceData_v2"    # AJUSTE: onde está a pasta (sem ~)
temp_folder  <- "/content/tax4fun2_output/Ref100R"                      # pasta de trabalho/temporários
db_mode      <- "Ref100NR"
threads      <- 2

library(Tax4Fun2) #import library

# 1. Run the reference blast
runRefBlast(path_to_otus = fasta_file,
            path_to_reference_data = ref_db_root,
            path_to_temp_folder = temp_folder,
            database_mode = db_mode,
            use_force = T, num_threads = 2)

# 2) makeFunctionalPrediction
cat("\n🧠 makeFunctionalPrediction...\n")
makeFunctionalPrediction(path_to_otu_table = otu_tsv,
                         path_to_reference_data = ref_db_root,
                         path_to_temp_folder = temp_folder,
                         database_mode = db_mode,
                         normalize_by_copy_number = TRUE,
                         min_identity_to_reference = 0.99, normalize_pathways = TRUE)

# 3) calculateFunctionalRedundancy (experimental)
cat("\n🧮 calculateFunctionalRedundancy...\n")

#Step 4: Calculating (multi-)functional redundancy indices (experimental)
#calculates phylogentic distributions of KEGG functions (High FRI -> high redundancy, low FRI -> function is less redundant and #might get lost with community change)

calculateFunctionalRedundancy(path_to_otu_table = otu_tsv,
                              path_to_reference_data = ref_db_root,
                              path_to_temp_folder = temp_folder,
                              database_mode = db_mode,
                              min_identity_to_reference = 0.99)



🧠 makeFunctionalPrediction...





 


A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages 

 




Error in FUN(X[[i]], ...) : invalid 'type' (character) of argument


RInterpreterError: Failed to parse and evaluate line '# --- CONFIGURAÇÃO DE CAMINHOS ---\n# Ajuste estes caminhos conforme os arquivos que você tem em /content\notu_tsv      <- "/content/tax4fun2_output/otu_table_for_tax4fun2.txt"        # sua OTU table em TSV\nfasta_file   <- "/content/tax4fun2_output/sequences.fasta"          # seu FASTA (seqs representativas)\nref_db_root  <- "/content/tax4fun2_output/Tax4Fun2_ReferenceData_v2"    # AJUSTE: onde está a pasta (sem ~)\ntemp_folder  <- "/content/tax4fun2_output/Ref100R"                      # pasta de trabalho/temporários\ndb_mode      <- "Ref100NR"\nthreads      <- 2\n\nlibrary(Tax4Fun2) #import library\n\n# 1. Run the reference blast\nrunRefBlast(path_to_otus = fasta_file, \n            path_to_reference_data = ref_db_root, \n            path_to_temp_folder = temp_folder, \n            database_mode = db_mode,\n            use_force = T, num_threads = 2)\n\n# 2) makeFunctionalPrediction\ncat("\\n🧠 makeFunctionalPrediction...\\n")\nma

#==============================================================
## ÉTAPE 6 - Bonus: Visualisation des Voies Métaboliques avec PathView
#==============================================================

In [ ]:
print('🗺️  Étape 6 : Visualisation des Voies Métaboliques KEGG')
print('='*60)
print('\nNous allons visualiser les voies métaboliques prédites par Tax4Fun2')
print('en utilisant PathView pour créer des cartes métaboliques colorées.')
print('\nVoies à explorer:')
print('  1. Cycle de l\'azote (Nitrogen metabolism)')
print('  2. Cycle du carbone (Carbon metabolism)')
print('  3. Dégradation des composés organiques')

In [ ]:
print('\n📦 Installation de PathView et dépendances...')

import rpy2.robjects as robjects
r = robjects.r

r('''
# Installer BiocManager si nécessaire
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager", quiet = TRUE)

# Installer PathView et packages nécessaires
BiocManager::install(c("pathview", "KEGGgraph", "KEGGREST"), quiet = TRUE, update = FALSE)

# Installer packages de visualisation
install.packages(c("RColorBrewer", "viridis"), quiet = TRUE)

print("✅ PathView et dépendances installés")
''')

### Préparation des données par champ


In [ ]:
import pandas as pd
import numpy as np

# Charger les résultats Tax4Fun2
results_file = 'tax4fun2_output/results/functional_predictions.txt'

if os.path.exists(results_file):
    df_functions = pd.read_csv(results_file, sep='\t', index_col=0)
    print(f'✅ Données chargées : {df_functions.shape[0]} fonctions × {df_functions.shape[1]} échantillons')

    # Charger les métadonnées de solo para identificar os campos
    soil_file = 'AtelierSYMBIOSOL/data/data_Tax4Fun2/soil_properties_cranberry.csv'
    df_soil = pd.read_csv(soil_file)

    # Renomear primeira coluna
    df_soil = df_soil.rename(columns={df_soil.columns[0]: 'SampleID'})

    # Harmonizar IDs (LEM59-01 → LEM5916S-01)
    df_soil['SampleID_16S'] = df_soil['SampleID'].str.replace('LEM59-', 'LEM5916S-', regex=False)

    print(f'\n📋 Métadonnées du sol : {df_soil.shape[0]} échantillons')
    print(f'   Champs disponibles : {df_soil["Campo"].unique() if "Campo" in df_soil.columns else "Non défini"}')

    # Agrupar amostras por campo
    if 'Campo' in df_soil.columns:
        # Criar dicionário de amostras por campo
        samples_by_field = {}
        for field in df_soil['Campo'].unique():
            field_samples = df_soil[df_soil['Campo'] == field]['SampleID_16S'].tolist()
            # Filtrar apenas amostras que existem nos resultados
            field_samples = [s for s in field_samples if s in df_functions.columns]
            samples_by_field[field] = field_samples
            print(f'   Campo {field}: {len(field_samples)} échantillons')

        # Calcular média por campo
        df_by_field = pd.DataFrame()
        for field, samples in samples_by_field.items():
            if samples:
                df_by_field[f'Campo_{field}'] = df_functions[samples].mean(axis=1)

        print(f'\n✅ Données agrégées par champ : {df_by_field.shape}')
        display(df_by_field.head())
    else:
        print('\n⚠️  Colonne "Campo" non trouvée, utilisation de tous les échantillons')
        df_by_field = df_functions
else:
    print(f'❌ Fichier de résultats non trouvé: {results_file}')

In [ ]:
# ============================================================================
# Visualisation avec PathView - Cycle de l'Azote
# ============================================================================

print('\n🗺️  Visualisation du Cycle de l\'Azote (KEGG pathway 00910)...')

# Sauvegarder dados para R
df_by_field.to_csv('tax4fun2_output/functions_by_field.txt', sep='\t')

r('''
library(pathview)
library(RColorBrewer)

# Charger les données
func_data <- read.table("tax4fun2_output/functions_by_field.txt",
                        header = TRUE, sep = "\\t", row.names = 1)

print(paste("Données chargées:", nrow(func_data), "fonctions"))

# Créer le répertoire pour les résultats PathView
dir.create("tax4fun2_output/pathview_results", showWarnings = FALSE)
setwd("tax4fun2_output/pathview_results")

# Pathway IDs pour les cycles importants
pathways <- list(
    "Nitrogen_metabolism" = "00910",
    "Carbon_metabolism" = "01200",
    "Methane_metabolism" = "00680",
    "Sulfur_metabolism" = "00920"
)

print("\\n🗺️  Génération des cartes métaboliques...")

# Générer PathView pour chaque voie
for (pathway_name in names(pathways)) {
    pathway_id <- pathways[[pathway_name]]

    tryCatch({
        print(paste("\\n   Génération de:", pathway_name, "(", pathway_id, ")"))

        # Générer PathView pour chaque campo
        for (campo in colnames(func_data)) {
            # Extrair dados de um campo
            campo_data <- func_data[, campo, drop = FALSE]

            # Remover NAs e zeros
            campo_data <- campo_data[!is.na(campo_data[,1]) & campo_data[,1] > 0, , drop = FALSE]

            if (nrow(campo_data) > 0) {
                # Gerar PathView
                pathview(
                    gene.data = campo_data,
                    pathway.id = pathway_id,
                    species = "ko",  # KEGG Orthology
                    out.suffix = paste0(pathway_name, "_", campo),
                    kegg.native = TRUE,
                    limit = list(gene = c(0, max(campo_data, na.rm = TRUE))),
                    low = "white",
                    mid = "yellow",
                    high = "red"
                )

                print(paste("      ✅", campo, "généré"))
            }
        }

    }, error = function(e) {
        print(paste("      ⚠️  Erreur pour", pathway_name, ":", e$message))
    })
}

print("\\n✅ Cartes métaboliques générées!")

# Retourner au diretório original
setwd("../..")

# Lister les fichiers générés
list.files("tax4fun2_output/pathview_results", pattern = ".png$")
''')

print('\n📂 Cartes métaboliques générées dans: tax4fun2_output/pathview_results/')
!ls -lh tax4fun2_output/pathview_results/*.png

In [ ]:
# ============================================================================
# PARTIE 4 : Affichage des cartes métaboliques
# ============================================================================

print('\n🖼️  Affichage des cartes métaboliques...')

from IPython.display import Image, display
import glob

# Trouver toutes as imagens PNG geradas
pathway_images = glob.glob('tax4fun2_output/pathview_results/*.png')

if pathway_images:
    print(f'\n✅ {len(pathway_images)} cartes trouvées')

    # Organizar por tipo de pathway
    nitrogen_imgs = [img for img in pathway_images if 'Nitrogen' in img]
    carbon_imgs = [img for img in pathway_images if 'Carbon' in img]

    if nitrogen_imgs:
        print('\n🔵 Cycle de l\'Azote:')
        for img in nitrogen_imgs:
            print(f'\n   {os.path.basename(img)}')
            display(Image(filename=img, width=800))

    if carbon_imgs:
        print('\n🟢 Métabolisme du Carbone:')
        for img in carbon_imgs:
            print(f'\n   {os.path.basename(img)}')
            display(Image(filename=img, width=800))
else:
    print('⚠️  Aucune carte générée')

In [ ]:
# ============================================================================
# PARTIE 5 : Analyse comparative entre champs
# ============================================================================

print('\n📊 Analyse comparative des fonctions entre champs...')

import matplotlib.pyplot as plt
import seaborn as sns

# Sélectionner fonctions liées au cycle de l'azote
nitrogen_functions = [
    'K00370',  # Nitrate reductase
    'K00371',  # Nitrite reductase
    'K00368',  # Nitrogenase
    'K02305',  # Nitrite reductase (NO-forming)
    'K00531'   # Hydroxylamine reductase
]

# Filtrar funções disponíveis
available_n_funcs = [f for f in nitrogen_functions if f in df_by_field.index]

if available_n_funcs:
    df_nitrogen = df_by_field.loc[available_n_funcs]

    # Criar heatmap
    plt.figure(figsize=(10, 6))
    sns.heatmap(df_nitrogen,
                annot=True,
                fmt='.2f',
                cmap='YlOrRd',
                cbar_kws={'label': 'Abondance relative'})
    plt.title('Fonctions du Cycle de l\'Azote par Champ',
              fontsize=14, fontweight='bold')
    plt.xlabel('Champ', fontsize=12)
    plt.ylabel('Fonction KEGG', fontsize=12)
    plt.tight_layout()
    plt.savefig('tax4fun2_output/nitrogen_functions_heatmap.png', dpi=300, bbox_inches='tight')
    plt.show()

    print('✅ Heatmap sauvegardé: tax4fun2_output/nitrogen_functions_heatmap.png')
else:
    print('⚠️  Fonctions du cycle de l\'azote non trouvées dans les résultats')

In [ ]:
# ============================================================================
# Visualisation moderne avec ggkegg (Alternative à PathView)
# ============================================================================

print('\n🎨 Alternative moderne : Visualisation avec ggkegg...')

r('''
# Installer ggkegg (package moderne pour visualisation KEGG)
if (!requireNamespace("remotes", quietly = TRUE))
    install.packages("remotes", quiet = TRUE)

remotes::install_github("noriakis/ggkegg", quiet = TRUE)

library(ggkegg)
library(ggplot2)

print("\\n📊 Création d'une visualisation moderne du cycle de l'azote...")

tryCatch({
    # Télécharger le pathway
    pathway <- pathway("ko00910")  # Nitrogen metabolism

    # Créer la visualisation
    p <- ggraph(pathway) +
         geom_node_rect(aes(fill = type)) +
         geom_edge_link(arrow = arrow(length = unit(1, "mm"))) +
         theme_void() +
         labs(title = "Cycle de l'Azote - Visualisation Moderne")

    ggsave("tax4fun2_output/nitrogen_cycle_modern.png",
           plot = p, width = 12, height = 8, dpi = 300)

    print("✅ Visualisation moderne créée!")

}, error = function(e) {
    print(paste("⚠️  ggkegg non disponible:", e$message))
    print("   (PathView reste la meilleure option)")
})
''')

## Conclusion

🎉 **Félicitations !** Vous avez terminé cet atelier.

### Ce que vous avez appris :
- Comment installer et configurer Tax4Fun2 dans Google Colab.
- Comment préparer les données QIIME2 pour Tax4Fun2.
- Comment exécuter Tax4Fun2 pour prédire les fonctions métaboliques.
- La différence entre l'analyse de guildes (FUNGuild) et l'analyse fonctionnelle (Tax4Fun2).

### Pour aller plus loin :
- Comparer les profils fonctionnels entre différents types d'échantillons.
- Corréler les fonctions métaboliques avec les propriétés du sol.
- Explorer les voies métaboliques spécifiques (ex: cycle de l'azote, du carbone).